## AG2 - ACTIVIDAD GUIADA 2
### Asignatura: Algoritmos de Optimización
### Laura Katherine Henao González

Repositorio GitHub:

In [17]:
!pip install requests
!pip install tsplib95

Carga de datos del problema

In [19]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos



In [20]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [21]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

15

Funciones básicas

In [22]:
#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

Búsqueda aleatoria

In [23]:

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)

Mejor solución: [0, 5, 27, 35, 21, 9, 13, 36, 6, 33, 3, 23, 40, 24, 30, 26, 11, 12, 8, 34, 20, 7, 16, 18, 10, 17, 37, 14, 15, 19, 4, 32, 38, 31, 29, 39, 41, 22, 1, 28, 25, 2]
Distancia     : 3777


Busqueda local

In [24]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 3777
Distancia Mejor Solucion Local: 3412


In [25]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

En la iteracion  36 , la mejor solución encontrada es: [0, 1, 7, 14, 16, 15, 37, 17, 36, 35, 31, 9, 23, 41, 18, 26, 5, 19, 13, 12, 11, 25, 10, 8, 2, 27, 38, 22, 24, 40, 21, 39, 29, 28, 30, 34, 33, 20, 32, 3, 4, 6]
Distancia     : 1715


Multi arranque

In [26]:
def busqueda_local(problem, num_intentos=10):
    mejor_solucion = []
    mejor_distancia = float('inf')  # inicializa con infinito

    for _ in range(num_intentos):
        solucion_referencia = crear_solucion(Nodos)
        while True:
            vecina = genera_vecina(solucion_referencia)

            distancia_vecina = distancia_total(vecina, problem)

            if distancia_vecina < mejor_distancia:
                mejor_solucion = vecina
                mejor_distancia = distancia_vecina
            else:
                break

            solucion_referencia = vecina

        print("En el intento", _+1, ", la mejor solución encontrada es:" , mejor_solucion)
        print("Distancia     :" , mejor_distancia)

    return mejor_solucion

sol = busqueda_local(problem)


En el intento 1 , la mejor solución encontrada es: [0, 7, 16, 14, 9, 41, 23, 40, 24, 21, 39, 34, 33, 38, 22, 30, 29, 28, 27, 3, 2, 8, 10, 25, 11, 12, 18, 26, 6, 1, 31, 17, 37, 15, 36, 35, 20, 32, 4, 5, 13, 19]
Distancia     : 1872
En el intento 2 , la mejor solución encontrada es: [0, 7, 16, 14, 9, 41, 23, 40, 24, 21, 39, 34, 33, 38, 22, 30, 29, 28, 27, 3, 2, 8, 10, 25, 11, 12, 18, 26, 6, 1, 31, 17, 37, 15, 36, 35, 20, 32, 4, 5, 13, 19]
Distancia     : 1872
En el intento 3 , la mejor solución encontrada es: [0, 7, 16, 14, 9, 41, 23, 40, 24, 21, 39, 34, 33, 38, 22, 30, 29, 28, 27, 3, 2, 8, 10, 25, 11, 12, 18, 26, 6, 1, 31, 17, 37, 15, 36, 35, 20, 32, 4, 5, 13, 19]
Distancia     : 1872
En el intento 4 , la mejor solución encontrada es: [0, 7, 16, 14, 9, 41, 23, 40, 24, 21, 39, 34, 33, 38, 22, 30, 29, 28, 27, 3, 2, 8, 10, 25, 11, 12, 18, 26, 6, 1, 31, 17, 37, 15, 36, 35, 20, 32, 4, 5, 13, 19]
Distancia     : 1872
En el intento 5 , la mejor solución encontrada es: [0, 7, 16, 14, 9, 41, 23,

Entornos Variables

In [27]:
# Modificación a funcion vecina
def genera_vecina(solucion, vecindario=1):
    mejor_solucion = solucion[:]
    mejor_distancia = distancia_total(solucion, problem)

    for _ in range(vecindario):
        for i in range(1, len(solucion)-1):
            for j in range(i+1, len(solucion)):
                vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
                distancia_vecina = distancia_total(vecina, problem)
                if distancia_vecina < mejor_distancia:
                    mejor_distancia = distancia_vecina
                    mejor_solucion = vecina

    return mejor_solucion


In [28]:
def busqueda_local(problem, max_vecindario=10):
    mejor_solucion = []
    solucion_referencia = crear_solucion(Nodos)
    mejor_distancia = distancia_total(solucion_referencia, problem)
    iteracion = 0
    vecindario = 1

    while True:
        iteracion += 1
        vecina = genera_vecina(solucion_referencia, vecindario)
        distancia_vecina = distancia_total(vecina, problem)

        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
        else:
            vecindario += 1
            if vecindario > max_vecindario:
                break

        solucion_referencia = vecina

    print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
    print("Distancia     :" , mejor_distancia)
    return mejor_solucion

sol = busqueda_local(problem)


En la iteracion  42 , la mejor solución encontrada es: [0, 28, 30, 38, 22, 29, 14, 16, 15, 17, 31, 36, 35, 39, 24, 40, 21, 9, 8, 34, 33, 20, 32, 4, 6, 26, 5, 19, 13, 10, 23, 41, 25, 11, 12, 18, 2, 27, 3, 1, 37, 7]
Distancia     : 1953
